In [1]:
import pandas as pd
import numpy as np
# from numba import njit

import matplotlib.pyplot as plt
from matplotlib import cm
from numpy import linalg as LA
from tqdm import tqdm

from kernel import *

from kernel_functions import * 
from preprocessing import preprocessing
from collections import defaultdict

from scipy.linalg import eigh


In [4]:
def linear_kernel(x, z):
    K = np.dot(x, z.T)
    return K

def gaussian_kernel(x, z, gamma):
   
    # s1 = x.shape[0]
    # s2 = z.shape[0]
    # xx = np.repeat((LA.norm(x, axis=1)**2).reshape((-1, 1)), s2, axis=1)
    # zz = np.repeat((LA.norm(z, axis=1)**2).reshape((-1, 1)), s1, axis=1)
    # K_tmp = np.exp(2 * linear_kernel(x, z) + xx + zz.T / (gamma**2))
    # K_tmp = np.exp((2 * linear_kernel(x, z) + xx + zz.T) / (gamma**2))
    # K = np.array([np.exp(LA.norm(x - z1, axis=1)**2/(gamma**2)) for z1 in z])
    D = l2distance(x, z)
    K_tmp = np.array(np.exp(D**2/(gamma**2)))
    return K_tmp

def fit(K, y, lamb = 0.1): 
    y = (y - 0.5) * 2
    # We solve the Dual
    NUM = K.shape[0]
    P = matrix(2 * K * y.reshape((-1,1)).dot(y.reshape((1,-1))))
    q = matrix(-np.ones((NUM, 1)))
    G = matrix(np.concatenate((-np.eye(NUM), np.eye(NUM)), axis=0))
    h = matrix(np.concatenate((np.zeros(NUM), lamb* np.ones(NUM)),axis=0))
    A = matrix(y.reshape(1, -1))
    b = matrix(np.zeros(1))
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x']) * y[:, None]
    bias = np.mean(y - np.dot(K, alphas))
    return alphas, bias


def predict(alphas, bias, K_test):
    mat = np.dot(K_test, alphas)
    mat = mat + bias>0.
    return mat.reshape(-1)

def testing_lambda(X_train, Y_train, X_test, Y_test, lamb=0.1, kernel='linear', gamma=1000, Kernels=None):
    if kernel=='linear':
        K = linear_kernel(X_train, X_train)
        K_test = linear_kernel(X_test, X_train)
    if kernel=='gaussian':
        K = gaussian_kernel(X_train, X_train, gamma)
        K_test = gaussian_kernel(X_test, X_train, gamma)
    if kernel=='custom':
        K = Kernels[0]
        K_test = Kernels[1]
        
    alphas, bias = fit(K, Y_train, lamb=lamb)
    
    Y_pred = predict(alphas, bias, K_test)
    #Y_test = (Y_test - 0.5) * 2
    acc_test = np.sum(Y_pred == Y_test)/Y_test.shape[0]
    
    Y_pred_train = predict(alphas, bias, K)
    acc_train = np.sum(Y_pred_train == Y_train)/X_train.shape[0]
    
    if np.alltrue(Y_pred==1):
        print("test Toute les valeurs sont TRUE")
    
    if np.alltrue(Y_pred==-1):
        print("Toute les valeurs sont FALSE")
    
    
    return acc_train, acc_test



In [3]:
# load all data as the numpy array type
#X = pd.read_csv('data/Xtr1_mat50.csv', sep=' ', header=None).values
X_raw0 = pd.read_csv('data/Xtr0.csv', sep= ' ', header = None).values.reshape((-1))
X_raw1 = pd.read_csv('data/Xtr1.csv', sep=' ', header=None).values.reshape((-1))
X_raw2 = pd.read_csv('data/Xtr2.csv', sep=' ', header=None).values.reshape((-1))

# transform k-fold
X_0 = to_k_fold(X_raw0, fold1=9, fold2 = 32)
X_1 = to_k_fold(X_raw1, fold1=9, fold2 = 32)
X_2 = to_k_fold(X_raw2, fold1=9, fold2 = 32)

# transform to an array of string
X_valid0 = pd.read_csv('data/Xte0.csv', sep=' ', header=None).values.reshape((-1))
X_valid1 = pd.read_csv('data/Xte1.csv', sep=' ', header=None).values.reshape((-1))
X_valid2 = pd.read_csv('data/Xte1.csv', sep=' ', header=None).values.reshape((-1))


Y0 = pd.read_csv('data/Ytr0.csv', sep=',', header=0)['Bound'].values
Y1 = pd.read_csv('data/Ytr1.csv', sep=',', header=0)['Bound'].values
Y2 = pd.read_csv('data/Ytr2.csv', sep=',', header=0)['Bound'].values

#print('numerical features shape', X.shape)
#print('numerical features first row', X[0])
# print('sequences shape: ', X_raw0.shape)
# print('sequence first row: ', X_raw0[0])
# print('labels shape', Y0.shape)

100%|██████████| 2000/2000 [00:19<00:00, 100.19it/s]


In [5]:
print(X_0.shape)

(2000, 1024)


In [6]:
X_0_t = to_k_fold(X_raw0[:], fold1=5, fold2 = 62)
print(X_0.shape)

100%|██████████| 2000/2000 [01:06<00:00, 29.97it/s]


(2000, 1024)


In [7]:
# bias_term = 1e6
gamma = 1000
X_train, Y_train, X_test, Y_test = preprocessing(X_0, Y0, percent=0.8, seed=2)
# K = gaussian_kernel(X_train, X_train, gamma)
# K_test = gaussian_kernel(X_test, X_train, gamma)
# print("kernel constructed")

for lamb in [1e-5, 2e-5, 3e-5]:
    acc_train, acc_test = testing_lambda(X_train, Y_train, X_test, Y_test, 
                                         lamb=lamb, gamma = 1000,
                                         kernel='linear'#, Kernels = (K, K_test) 
                                        )
    print("lamb = {}, acc train = {}, acc_test = {}".format(lamb, acc_train, acc_test))

lamb = 1e-05, acc train = 0.84, acc_test = 0.7375
lamb = 2e-05, acc train = 0.865625, acc_test = 0.72
lamb = 3e-05, acc train = 0.880625, acc_test = 0.715


In [6]:
X_train, Y_train, X_test, Y_test = preprocessing(X_1, Y1, percent=0.8)
for lamb in np.arange(1e-6, 1e-5, 1e-6):
    acc_train, acc_test = testing_lambda(X_train, Y_train, X_test, Y_test, lamb=lamb)
    print("lamb = {}, acc train = {}, acc_test = {}".format(lamb, acc_train, acc_test))

lamb = 1e-06, acc train = 0.754375, acc_test = 0.745
lamb = 2e-06, acc train = 0.765, acc_test = 0.7475
lamb = 3e-06, acc train = 0.78875, acc_test = 0.7725
lamb = 4e-06, acc train = 0.80625, acc_test = 0.785
lamb = 4.9999999999999996e-06, acc train = 0.836875, acc_test = 0.795
lamb = 5.999999999999999e-06, acc train = 0.851875, acc_test = 0.7925
lamb = 7e-06, acc train = 0.863125, acc_test = 0.7975
lamb = 8e-06, acc train = 0.8725, acc_test = 0.8075
lamb = 9e-06, acc train = 0.884375, acc_test = 0.8075


In [66]:
np.sum(Y0)/Y0.shape[0]

0.5

In [121]:
X_train, Y_train, X_test, Y_test = preprocessing(X_tmp2, Y2, percent=0.9)
for lamb in [1e-4, 1e-3, 1e-2]:
    acc_train, acc_test = testing_lambda(X_train, Y_train, X_test, Y_test, lamb=lamb)
    print("lamb = {}, acc train = {}, acc_test = {}".format(lamb, acc_train, acc_test))

lamb = 0.0001, acc train = 0.8872222222222222, acc_test = 0.495
lamb = 0.001, acc train = 0.9861111111111112, acc_test = 0.625
lamb = 0.01, acc train = 1.0, acc_test = 0.605


In [68]:
np.sum(Y2)/Y2.shape[0]

0.5

0.5

## TESTING


In [27]:
def test(X_train, Y_train, X_test, lamb=1.):
    # X_train, Y_train, X_test, Y_test = preprocessing(X, Y, percent=0.8)
    X_train_preprocess = np.concatenate((X_train, np.ones((X_train.shape[0], 1))), axis=1)
    X_test_preprocess = np.concatenate((X_test, np.ones((X_test.shape[0], 1))), axis=1)
    K = X_train_preprocess.dot(X_train_preprocess.T)
    K_test = X_test_preprocess.dot(X_train_preprocess.T)
    w = solve_svm(K, Y_train, lamb=lamb, kktreg = 1e-9)
    n = K.shape[0]
    Y_predicted = np.dot(K_test, w[:n]) > 0.
    Y_predicted = Y_predicted + 0.0
    # result = ((Y_test+1.)/ 2. == np.transpose(Y_predicted))
    Y_predicted_train = np.dot(K, w[:n]) > 0.
    result_train = ((Y_train+1)/ 2 == np.transpose(Y_predicted_train))
    if np.alltrue(Y_predicted):
        print("Toute les valeurs sont TRUE")
    if np.alltrue(Y_predicted==False):
        print("Toute les valeurs sont FALSE")
    print("lambda = {}".format(lamb))
    print("train : {}".format(np.mean(result_train)))
    return Y_predicted

In [22]:
X_test0 = to_k_fold(X_valid0, fold1=9, fold2 = 64)
X_test1 = to_k_fold(X_valid1, fold1=9, fold2 = 64)
X_test2 = to_k_fold(X_valid2, fold1=9, fold2 = 64)

100%|██████████| 1000/1000 [00:31<00:00, 31.45it/s]


In [28]:
Y0_t = (Y0 - 0.5) *2
w, bias = fit(X_0, Y0_t, lamb=1.)
Y_pred0 = predict(w, bias, X_test0)

     pcost       dcost       gap    pres   dres
 0:  9.2621e+00  1.2864e+01  5e+03  2e+00  3e+06
 1:  7.2937e+00 -2.8847e+02  3e+02  1e-01  2e+05
 2:  2.5566e+00 -5.0199e+01  5e+01  2e-02  3e+04
 3:  1.6728e+00 -5.7320e+00  7e+00  2e-03  3e+03
 4:  1.3318e+00  8.2074e-02  1e+00  2e-04  3e+02
 5:  6.3867e-01  5.2630e-01  1e-01  4e-06  8e+00
 6:  5.9189e-01  5.6471e-01  3e-02  9e-07  2e+00
 7:  5.8054e-01  5.7490e-01  6e-03  1e-07  2e-01
 8:  5.7782e-01  5.7717e-01  7e-04  9e-09  2e-02
 9:  5.7752e-01  5.7743e-01  9e-05  1e-09  2e-03
10:  5.7747e-01  5.7747e-01  4e-06  7e-10  7e-05
11:  5.7747e-01  5.7747e-01  8e-08  7e-10  1e-06
12:  5.7747e-01  5.7747e-01  2e-09  7e-10  2e-08
Optimal solution found.
lambda = 1.5
train : 0.833


In [29]:
Y1_t = (Y1 - 0.5) *2
w, bias = fit(X_1, Y1_t, lamb=1.)
Y_pred1 = predict(w, bias, X_test1)

     pcost       dcost       gap    pres   dres
 0:  6.9083e+00  1.0034e+01  5e+03  2e+00  2e+07
 1:  5.5102e+00 -2.2544e+02  2e+02  8e-02  1e+06
 2:  1.8893e+00 -2.8892e+01  3e+01  9e-03  1e+05
 3:  1.4867e+00 -3.5336e+00  5e+00  1e-03  2e+04
 4:  1.1001e+00 -8.0780e-02  1e+00  2e-04  2e+03
 5:  4.9247e-01  3.3941e-01  2e-01  1e-05  2e+02
 6:  4.2388e-01  3.8429e-01  4e-02  3e-06  4e+01
 7:  4.0657e-01  3.9766e-01  9e-03  5e-07  7e+00
 8:  4.0255e-01  4.0082e-01  2e-03  8e-08  1e+00
 9:  4.0166e-01  4.0152e-01  1e-04  5e-09  8e-02
10:  4.0159e-01  4.0158e-01  7e-06  7e-10  3e-03
11:  4.0159e-01  4.0159e-01  3e-07  8e-10  1e-04
12:  4.0159e-01  4.0159e-01  8e-09  8e-10  2e-06
13:  4.0159e-01  4.0159e-01  3e-10  8e-10  2e-08
Optimal solution found.
lambda = 2.0
train : 0.9205


In [30]:
Y2_t = (Y2 - 0.5) *2
w, bias = fit(X_2, Y2_t, lamb=1.)
Y_pred2 = predict(w, bias, X_test2)

     pcost       dcost       gap    pres   dres
 0:  1.5420e+01  2.1448e+01  5e+03  2e+00  6e+06
 1:  1.0812e+01 -3.7609e+02  4e+02  1e-01  5e+05
 2:  2.9521e+00 -5.2604e+01  6e+01  2e-02  6e+04
 3:  1.8370e+00 -6.9842e+00  9e+00  2e-03  8e+03
 4:  1.5149e+00  1.9806e-01  1e+00  1e-04  5e+02
 5:  8.2708e-01  6.2501e-01  2e-01  2e-05  7e+01
 6:  7.2171e-01  6.8425e-01  4e-02  3e-06  1e+01
 7:  7.0418e-01  6.9742e-01  7e-03  3e-07  1e+00
 8:  7.0081e-01  7.0005e-01  8e-04  3e-08  1e-01
 9:  7.0044e-01  7.0035e-01  9e-05  3e-09  1e-02
10:  7.0039e-01  7.0039e-01  6e-06  7e-10  7e-04
11:  7.0039e-01  7.0039e-01  2e-07  8e-10  2e-05
12:  7.0039e-01  7.0039e-01  5e-09  8e-10  3e-07
13:  7.0039e-01  7.0039e-01  2e-10  8e-10  3e-09
Optimal solution found.
lambda = 1.8
train : 0.769


In [33]:
test0 = np.transpose(Y_pred0)[:][0]
test1 = np.transpose(Y_pred1)[:][0]
test2 = np.transpose(Y_pred2)[:][0]

bound = np.concatenate((test0,test1,test2), axis=0).reshape((-1)).astype(int)
final = pd.DataFrame(np.arange(3000), columns=['Id'])
final['Bound'] = bound
final.to_csv('resultk_fold.csv', index= None)

## Another

In [105]:

from scipy.spatial.distance import cdist

class LinearKernel():

	def __init__(self):
		self.kernel = 'linear'

	def K_matrix(self, X1, X2=None):
		if X2 is None:
			X2 = X1
		return np.dot(X1, X2.T)
    


class GaussianKernel():

	def __init__(self, sigma=1.):
		self.kernel = 'gaussian'
		self.sigma = sigma

	def K_matrix(self, X1, X2=None):
		if X2 is None:
			X2 = X1
		K = cdist(X1, X2, 'sqeuclidean')
		K = np.exp(-K/(2*self.sigma**2))
		return K

In [106]:
KernelMapping = {
    'linear':LinearKernel,
    'gaussian':GaussianKernel
}


def sigmoid(z):
    return 1/(1+np.exp(-z))

class KernelLogisticRegression():

    def __init__(self, kernel='linear', lamb=1e-2, maxiter=1e5, threshold=1e-3, **kwargs):
        """
        kernel specifies the type of kernel to use
        lamb specifies the regularization parameter
        """

        self.kernel = KernelMapping[kernel](**kwargs)
        self.lamb = lamb
        self.alpha = None
        self.dim = None
        self.X_ = None
        self.maxiter = maxiter
        self.threshold = threshold
        self.t = 0

    def update_alpha(self, K, Y):
        """
        itertive step to update alpha when training the classifier
        """
        n = K.shape[0]
        m = np.dot(K, self.alpha).reshape(-1)
        P = -sigmoid(-Y*m)
        W = sigmoid(m)*sigmoid(-m)
        z = m - P*Y/W

        del P, m
        W = np.diag(np.sqrt(W.reshape(-1)))

        A = np.dot(np.dot(W, K), W) + n*self.lamb*np.eye(n)
        A = np.dot(np.dot(W, np.linalg.inv(A)), W)

        del W

        return np.dot(A, z)

    def fit(self, X, Y, verbose=False, return_pred=False):

        Y = 2*Y-1

        n = X.shape[0]
        self.dim = X.shape[1]

        if verbose:
            t0 = time()

        K = self.kernel.K_matrix(X)

        if verbose:
            print('Kernel matrix computed in {0:.2f} seconds'.format(time()-t0))

        
        diff = np.infty ## Convergence value
        if self.t==0:
            self.alpha = np.zeros(n)

        while (self.t < self.maxiter and diff > self.threshold):

            alpha_new = self.update_alpha(K, Y)

            diff = np.sum((self.alpha-alpha_new)**2)

            
            self.alpha = np.copy(alpha_new)

            if verbose:
                print('Update difference for alpha:', diff)

            self.t += 1

        self.X_ = X

        if return_pred:
            f = np.dot(K, self.alpha)
            pred = sigmoid(f)
            pred = (pred>0.5).astype(int)
            pred = 2*pred - 1
            return np.mean(Y==pred)


    def predict(self, X):
        K = self.kernel.K_matrix(X, self.X_)
        f = np.dot(K, self.alpha)
        pred = sigmoid(f)
        return (pred>0.5).astype(int)

    def score(self, X, Y):
        Y_test = self.predict(X)
        return np.mean(Y==Y_test)

In [107]:
for lamb in np.arange(1e4, 2e4, 1e3):
    log_kernel = KernelLogisticRegression(kernel='gaussian', lamb=lamb, threshold=1e-3, sigma=1000. )
    acc_train = log_kernel.fit(X_train, (Y_train+1.)/2., return_pred=True)
    acc_test = (log_kernel.score(X_test, (Y_test+1.)/2.))
    print("lamb = {}, acc train = {}, acc_test = {}".format(lamb, acc_train, acc_test))

lamb = 10000.0, acc train = 0.574375, acc_test = 0.5125
lamb = 11000.0, acc train = 0.574375, acc_test = 0.5125
lamb = 12000.0, acc train = 0.574375, acc_test = 0.5125
lamb = 13000.0, acc train = 0.574375, acc_test = 0.5125
lamb = 14000.0, acc train = 0.574375, acc_test = 0.5125
lamb = 15000.0, acc train = 0.574375, acc_test = 0.5125
lamb = 16000.0, acc train = 0.574375, acc_test = 0.5125
lamb = 17000.0, acc train = 0.574375, acc_test = 0.5125
lamb = 18000.0, acc train = 0.574375, acc_test = 0.5125


KeyboardInterrupt: 

In [93]:
def test_var_kwargs(farg, **kwargs):
    print("formal arg:", farg)
    for key in kwargs:
        print("another keyword arg: %s: %s" % (key, kwargs[key]))

test_var_kwargs(farg=1, myarg2="two", myarg3=3)


formal arg: 1
another keyword arg: myarg2: two
another keyword arg: myarg3: 3


In [ ]:


def count_kuplet_gap(seq, k_tmp, fold=5, k_grams_count=None):
    assert fold%2 == 1 or fold ==2, "fold must be odd"
    fold_size = len(bin(fold)[2:])
    fold = bin(fold)[2:]
    base_azote = ['A', 'C', 'T', 'G']

    if k_grams_count is None:
        k_grams_count = dict()

    tab = [''.join(i) for i in product(base_azote, repeat=np.sum([int(i) for i in fold]))]
    for code in tab:
        # print(''.join([code, '_', fold]))
        # k_grams_count[''.join([code, '_', fold])] = 0.
        # k_tmp[''.join([code, '_', fold])] = 0.
        k_grams_count[''.join([code, '_'])] = k_grams_count.get(''.join([code, '_']), 0.)
        k_tmp[''.join([code, '_'])] = k_tmp.get(''.join([code, '_']), 0.)

    for i in range(len(seq) - fold_size+1):
        l = seq[i:(i+fold_size)]
        # print(l)
        l = ''.join(l[i]*int(fold[i]) for i in range(len(fold)))
        # k_grams_count[''.join([l, '_', fold])] += 1./(len(seq) - fold_size+1)
        # k_tmp[''.join([l, '_', fold])] += 1.
        k_grams_count[''.join([l, '_'])] += 0.9**(len(seq) - fold_size+1) # 1./(len(seq) - fold_size+1)
        k_tmp[''.join([l, '_'])] += 1.
    return k_grams_count, k_tmp

def count_k_fold(sent, k_tmp, fold1 = 9, fold2 = 64):
    k_grams_count = dict()
    k_tmp = dict()
    for fold in np.arange(fold1, fold2, 2):
        if(np.sum([int(i) for i in bin(fold)[2:]]) in [7, 8]):
            k_grams_count, k_tmp = count_kuplet_gap(sent, k_tmp, fold = fold, k_grams_count=k_grams_count)
    return np.array([k_grams_count[key] for key in sorted(k_grams_count)]), np.array([k_tmp[key] for key in sorted(k_tmp)])


def to_k_fold(X, fold1=9, fold2=64):
    '''
    From X (X_raw) compute a sequence X_process using count_k_fold
    '''

    X_process = []
    X_count = []
    k_tmp = defaultdict(int)
    for sent in tqdm(X):
        sent_process, k_tmp = count_k_fold(sent, k_tmp, fold1 = fold1, fold2 = fold2)
        X_process.append(sent_process)
        X_count.append(k_tmp)

    X_process = np.array(X_process) * len(X) * len(X[0])
    sum_tmp = np.sum(np.array(X_count), axis=0)

    X_process = np.divide(X_process, sum_tmp, out=np.zeros_like(X_process), where=sum_tmp!=0)

    return X_process